## Importing the necessary libraries

In [10]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as gp

### Loading the census_data.csv from the data directory.
#### The code below: 
Checks for the existence of data/census_data.csv before attempting to read it. If the file is not found, it will print an error message indicating that the file does not exist. If the file is found, it will proceed to read the file into a DataFrame and display its information.

In [11]:
# Census data file path
file_path = 'data/census_data.csv'

# Check if the file exists
if os.path.isfile(file_path):
    # If the file exists, read the CSV file into a DataFrame
    census_df = pd.read_csv(file_path)
    print(".... CENSUS DATA LOADED ....\n")
    
    # Display information about the DataFrame
    census_df.info()
else:
    # If the file does not exist, print an error message
    print(f"Error: The file '{file_path}' does not exist.")

.... CENSUS DATA LOADED ....

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   House Number                   9426 non-null   int64  
 1   Street                         9426 non-null   object 
 2   First Name                     9426 non-null   object 
 3   Surname                        9426 non-null   object 
 4   Age                            9426 non-null   float64
 5   Relationship to Head of House  9426 non-null   object 
 6   Marital Status                 6996 non-null   object 
 7   Gender                         9426 non-null   object 
 8   Occupation                     9426 non-null   object 
 9   Infirmity                      9426 non-null   object 
 10  Religion                       6937 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 810.2+ KB


## Data Cleaning

To clean the census DataFrame for data analysis and predictive modeling, I am following this steps.

1. Handle Missing Values.

2. Convert Data Types: Ensure all columns have the appropriate data types.

3. Standardize Data: Ensure consistency in categorical data (e.g., make sure 'Gender' values are standardized).

4. Remove or Impute Outliers: Check for and handle outliers, especially in numerical data like Age.

### 1. Handle Missing Values

In [12]:
# Calculate the sum of missing entries for each column
census_df.isna().sum()

House Number                        0
Street                              0
First Name                          0
Surname                             0
Age                                 0
Relationship to Head of House       0
Marital Status                   2430
Gender                              0
Occupation                          0
Infirmity                           0
Religion                         2489
dtype: int64

### 

In [16]:
# Checking to see if they are entries with empty strings
empty_string_mask = census_df.applymap(lambda x: x == ' ').sum()

empty_string_mask

House Number                     0
Street                           0
First Name                       1
Surname                          0
Age                              0
Relationship to Head of House    0
Marital Status                   1
Gender                           0
Occupation                       1
Infirmity                        7
Religion                         1
dtype: int64

In [175]:
census_df['Marital Status'].unique()

array(['Married', nan, 'Divorced', 'Single', 'Widowed', ' '], dtype=object)

### -  Inspecting the unique entries in the Religion Column

In [176]:
# Filtering out the Religion column
census_df['Religion'].unique()

array(['None', nan, 'Catholic', 'Methodist', 'Christian', 'Muslim',
       'Private', 'Jewish', 'Sikh', 'Nope', 'Buddist', 'Baptist', 'Sith',
       'Bahai', 'Agnostic', 'Orthodoxy', ' '], dtype=object)

### - Deal with nan values in Marital Status and Religion.

In [177]:
# Fill nan values with a "UNKNOW" placeholder
census_df['Marital Status'].fillna('Unknown', inplace=True)

# Fill nan values in Religion to "UNKNOW"
census_df['Religion'].fillna('Unknown', inplace=True)

In [178]:
# checking for missing values
census_df.isna().sum()

House Number                     0
Street                           0
First Name                       0
Surname                          0
Age                              0
Relationship to Head of House    0
Marital Status                   0
Gender                           0
Occupation                       0
Infirmity                        0
Religion                         0
dtype: int64

In [179]:
census_df['Marital Status'].unique()

array(['Married', 'Unknown', 'Divorced', 'Single', 'Widowed', ' '],
      dtype=object)

In [180]:
census_df['Religion'].unique()

array(['None', 'Unknown', 'Catholic', 'Methodist', 'Christian', 'Muslim',
       'Private', 'Jewish', 'Sikh', 'Nope', 'Buddist', 'Baptist', 'Sith',
       'Bahai', 'Agnostic', 'Orthodoxy', ' '], dtype=object)

Above are the unique entries in the Marital status, Noticing that the marital status and Religion column both contains empty string(' ')

### Investigating the entry with the empty string (' ') in both the 'Marital Status' and 'Religion' Column

### -- Marital Status

Fecthing all entries with empty strings 

In [181]:
census_df.loc[census_df['Marital Status'] == ' ']

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
8941,4,Parsons Inlet,Jason,Davies,56.0,Husband,,Male,Geochemist,None,Catholic


#### 
using the information gotten from the entry above entry street, house number, and surname to investigate marital status 

In [182]:
census_df.loc[(census_df['Street'] == 'Parsons Inlet') & 
             (census_df['Surname'] == 'Davies') &
              (census_df['House Number'] == 4)
             ]

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
8940,4,Parsons Inlet,Heather,Davies,51.0,Head,Married,Female,Equality and diversity officer,None,Catholic
8941,4,Parsons Inlet,Jason,Davies,56.0,Husband,,Male,Geochemist,None,Catholic
8942,4,Parsons Inlet,Jacqueline,Davies,12.0,Daughter,Unknown,Female,Student,None,Unknown
8943,4,Parsons Inlet,Gavin,Davies,11.0,Son,Unknown,Male,Student,None,Unknown


#### Replacing the empty entry in the Marital Status with 'Married'

In [183]:
census_df['Marital Status'] = census_df['Marital Status'].replace(' ', 'Married')
census_df.loc[census_df['Marital Status'] == ' ']

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion


### -- Religion

In [184]:
census_df.loc[census_df['Religion'] == ' ']

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
8840,4,ExcaliburBells Street,Jean,O'Brien,43.0,Wife,Married,Female,Ecologist,None,


#### Replacing the empty entry in the Religion to 'Unknown'

In [185]:
census_df['Religion'] = census_df['Religion'].replace(' ', 'Unknown')

census_df['Religion'].unique()

array(['None', 'Unknown', 'Catholic', 'Methodist', 'Christian', 'Muslim',
       'Private', 'Jewish', 'Sikh', 'Nope', 'Buddist', 'Baptist', 'Sith',
       'Bahai', 'Agnostic', 'Orthodoxy'], dtype=object)

#### Replacing the 'Nope' entries to 'None'

In [186]:
census_df['Religion'] = census_df['Religion'].replace('Nope', 'None')

census_df['Religion'].unique()

array(['None', 'Unknown', 'Catholic', 'Methodist', 'Christian', 'Muslim',
       'Private', 'Jewish', 'Sikh', 'Buddist', 'Baptist', 'Sith', 'Bahai',
       'Agnostic', 'Orthodoxy'], dtype=object)

### 3.  Convert data types if necessary

In [187]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   House Number                   9426 non-null   int64  
 1   Street                         9426 non-null   object 
 2   First Name                     9426 non-null   object 
 3   Surname                        9426 non-null   object 
 4   Age                            9426 non-null   float64
 5   Relationship to Head of House  9426 non-null   object 
 6   Marital Status                 9426 non-null   object 
 7   Gender                         9426 non-null   object 
 8   Occupation                     9426 non-null   object 
 9   Infirmity                      9426 non-null   object 
 10  Religion                       9426 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 810.2+ KB


#### Ensuring that the Age column is converted to an integer.

In [188]:
# Convert 'Age' to int
census_df['Age'] = census_df['Age'].astype(int)

# Display the updated DataFrame information
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   House Number                   9426 non-null   int64 
 1   Street                         9426 non-null   object
 2   First Name                     9426 non-null   object
 3   Surname                        9426 non-null   object
 4   Age                            9426 non-null   int32 
 5   Relationship to Head of House  9426 non-null   object
 6   Marital Status                 9426 non-null   object
 7   Gender                         9426 non-null   object
 8   Occupation                     9426 non-null   object
 9   Infirmity                      9426 non-null   object
 10  Religion                       9426 non-null   object
dtypes: int32(1), int64(1), object(9)
memory usage: 773.4+ KB


In [189]:
census_df['Age']

0       36
1       30
2        0
3       40
4       31
        ..
9421    29
9422    29
9423    53
9424    54
9425    22
Name: Age, Length: 9426, dtype: int32

### 4.  Standardize Data: Ensure consistency in categorical data (e.g., make sure 'Gender' values are standardized).

In [190]:
census_df['Gender'] = census_df['Gender'].str.lower().str.capitalize()

In [191]:
census_df['Gender'].unique()

array(['Male', 'Female'], dtype=object)

### 5. Removing Un-necessary Columns

By focusing necessary columns, we retain the most relevant information for understanding the population and making meaningful predictions, while removing columns that do not contribute to these goals.

In [192]:
#
census_df

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
0,1,Bulldog Crescent,Harry,Baldwin,36,Head,Married,Male,Education administrator,None,None
1,1,Bulldog Crescent,Heather,Baldwin,30,Wife,Married,Female,Purchasing manager,None,None
2,1,Bulldog Crescent,Katy,Baldwin,0,Daughter,Unknown,Female,Child,None,Unknown
3,1,Bulldog Crescent,Philip,Giles,40,Lodger,Divorced,Male,Manufacturing engineer,None,None
4,1,Bulldog Crescent,Shannon,Allen,31,Lodger,Single,Female,Restaurant manager,None,Catholic
...,...,...,...,...,...,...,...,...,...,...,...
9421,1,Duchess Longhouse,Iain,Gardner,29,Son,Single,Male,"Designer, fashion/clothing",None,None
9422,1,Duchess Longhouse,Ashleigh,Gardner,29,Daughter,Single,Female,Farm manager,None,None
9423,1,Stephenson Barn,Anna,Reed,53,Head,Married,Female,Financial planner,None,Christian
9424,1,Stephenson Barn,Gregory,Reed,54,Husband,Married,Male,Naval architect,None,None


In [193]:
# List un-necessary columns
removing_columns = ['House Number', 'Street', 'First Name', 'Surname']

# Remove the columns from census_df
cleaned_df = census_df.drop(columns=removing_columns)

In [194]:
cleaned_df

,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
0,36,Head,Married,Male,Education administrator,None,None
1,30,Wife,Married,Female,Purchasing manager,None,None
2,0,Daughter,Unknown,Female,Child,None,Unknown
3,40,Lodger,Divorced,Male,Manufacturing engineer,None,None
4,31,Lodger,Single,Female,Restaurant manager,None,Catholic
...,...,...,...,...,...,...,...
9421,29,Son,Single,Male,"Designer, fashion/clothing",None,None
9422,29,Daughter,Single,Female,Farm manager,None,None
9423,53,Head,Married,Female,Financial planner,None,Christian
9424,54,Husband,Married,Male,Naval architect,None,None
